<table>
 <td>All code is written by Joo Won Park, Columbia University (jp3378@columbia.edu)</td>
</table>

In [2]:
#Loading libraries
import numpy as np
import IPython
from scipy.io.wavfile import read
from scipy.io.wavfile import write
from scipy import signal
import scipy.io as sio

* Load the desired audio file (mono) in place of 'drumloop.wav'. The file should be located in
the same folder as this python notebook file is. 

* Load the desired HRTF file from MARL NYU's repository in place of 'S002_marl-nyu.mat'. 
(which can be found [here](http://steinhardt.nyu.edu/marl/research/head_related_impulse_responses_repository))

* Running this code will produce 24 audio files. Each file will be convolved so that that the whole set of file pans the horizontal plane by 15 degrees. The names of file will be 'audioHRIR_0', ...,'audioHRIR_23',


In [3]:
Audio = 'drumloop.wav'
HRIRfile = 'S002_marl-nyu.mat'

audiofile = read(Audio)
audio_data, audio_f = audiofile[1], audiofile[0]

loadedfile = sio.loadmat(HRIRfile)
evlist = loadedfile['data']['elevation']
azlist = loadedfile['data']['azimuth']
impulses = loadedfile['data']['IR'][0]

ev0 = np.where(evlist==[[0]]) #index where 0 elevation 
azonly = azlist[ev0]
impulses0 = impulses[ev0[1]]



d={}
for i in xrange(0,len(azonly)):
    d["new_{0}".format(i)]=np.vstack((signal.convolve(audio_data,impulses0[i][:,0],mode='same'),
                                      signal.convolve(audio_data,impulses0[i][:,1],mode='same')))


def scaleandwrite(outputfile, samplerate, drumdata):
    scaled = np.int16((drumdata)/np.max(np.abs(drumdata)) * 32767)
    write(outputfile,rate = samplerate, data = scaled.T)

    
for i in xrange(0,len(azonly)):
    scaleandwrite('audioHRIR_{0}.wav'.format(i), drum_f, d['new_{0}'.format(i)])
    